In [1]:
!pip install trino

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 KB 873.1 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.4/136.4 KB 2.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 22.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 KB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.6/153.6 KB 38.3 MB/s eta 0:00:00


In [17]:
from trino.dbapi import connect

conn = connect(
    host="localhost",
    port=8082,
    user="admin"    # anything, required by Trino
)

cur = conn.cursor()

cur.execute("SHOW CATALOGS")
catalogs = cur.fetchall()
catalogs
print(catalogs)

# 2️⃣ Show all schemas to confirm
cur.execute("SHOW SCHEMAS FROM delta")
schemas = cur.fetchall()
print(schemas)

[['delta'], ['hive'], ['system']]
[['_control'], ['bronze2'], ['default'], ['hyb_bronze'], ['hyb_silver'], ['information_schema']]


In [6]:
# Import the Trino library and connect
import trino

# Connect to Trino
conn = trino.dbapi.connect(
    host='localhost',        # your Trino coordinator host
    port=8082,               # your Trino port
    user='your_username',    # Trino user
    catalog='delta',         # default catalog
    schema='default'         # default schema for initial connection
)

# Create a cursor to execute queries
cur = conn.cursor()

# 1️⃣ Create schemas if they don't exist
cur.execute("CREATE SCHEMA IF NOT EXISTS delta.hyb_bronze")

# 2️⃣ Show all schemas to confirm
cur.execute("SHOW SCHEMAS FROM delta")
schemas = cur.fetchall()
print(schemas)

# 3️⃣ Create a bronze table (if not exists)
cur.execute("""
CREATE TABLE IF NOT EXISTS delta.hyb_bronze.sample_bronze (
    id INT,
    name VARCHAR,
    value DOUBLE
)
""")

# 4️⃣ Insert some sample data
cur.execute("""
INSERT INTO delta.hyb_bronze.sample_bronze (id, name, value)
VALUES (1, 'sensor1', 12.3),
       (2, 'sensor2', 45.6)
""")

# 5️⃣ Query the table
cur.execute("SELECT * FROM delta.hyb_bronze.sample_bronze")
rows = cur.fetchall()
print(rows)

# ✅ Output should be something like:
# [(1, 'sensor1', 12.3), (2, 'sensor2', 45.6)]

# 1️⃣ Create silver schema if not exists
cur.execute("CREATE SCHEMA IF NOT EXISTS delta.hyb_silver")

# 2️⃣ Create a silver table (if not exists)
cur.execute("""
CREATE TABLE IF NOT EXISTS delta.hyb_silver.sample_silver (
    id INT,
    name VARCHAR,
    value DOUBLE
)
""")

# 3️⃣ Insert data from bronze to silver
# Here you can also transform or filter data if needed
cur.execute("""
INSERT INTO delta.hyb_silver.sample_silver (id, name, value)
SELECT id, name, value
FROM delta.hyb_bronze.sample_bronze
""")

# 4️⃣ Query the silver table to verify
cur.execute("SELECT * FROM delta.hyb_silver.sample_silver")
rows_silver = cur.fetchall()
print("Silver table data:", rows_silver)

import pandas as pd

# Query silver table
cur.execute("SELECT * FROM delta.hyb_silver.sample_silver")
rows_silver = cur.fetchall()

# Get column names
columns = [desc[0] for desc in cur.description]

# Create DataFrame
df_silver = pd.DataFrame(rows_silver, columns=columns)

# Display
df_silver



[['_control'], ['bronze2'], ['default'], ['hyb_bronze'], ['hyb_silver'], ['information_schema']]
[[1, 'sensor1', 12.3], [2, 'sensor2', 45.6], [1, 'sensor1', 12.3], [2, 'sensor2', 45.6], [1, 'sensor1', 12.3], [2, 'sensor2', 45.6], [1, 'sensor1', 12.3], [2, 'sensor2', 45.6], [1, 'sensor1', 12.3], [2, 'sensor2', 45.6]]
Silver table data: [[1, 'sensor1', 12.3], [2, 'sensor2', 45.6], [1, 'sensor1', 12.3], [2, 'sensor2', 45.6], [1, 'sensor1', 12.3], [2, 'sensor2', 45.6], [1, 'sensor1', 12.3], [2, 'sensor2', 45.6], [1, 'sensor1', 12.3], [2, 'sensor2', 45.6], [1, 'sensor1', 12.3], [2, 'sensor2', 45.6], [1, 'sensor1', 12.3], [2, 'sensor2', 45.6], [1, 'sensor1', 12.3], [2, 'sensor2', 45.6], [1, 'sensor1', 12.3], [2, 'sensor2', 45.6], [1, 'sensor1', 12.3], [1, 'sensor1', 12.3], [2, 'sensor2', 45.6], [1, 'sensor1', 12.3], [2, 'sensor2', 45.6], [2, 'sensor2', 45.6], [1, 'sensor1', 12.3], [2, 'sensor2', 45.6], [2, 'sensor2', 45.6], [1, 'sensor1', 12.3], [1, 'sensor1', 12.3], [2, 'sensor2', 45.6]]


id     name  value
0    1  sensor1   12.3
1    2  sensor2   45.6
2    1  sensor1   12.3
3    2  sensor2   45.6
4    1  sensor1   12.3
5    2  sensor2   45.6
6    1  sensor1   12.3
7    2  sensor2   45.6
8    1  sensor1   12.3
9    1  sensor1   12.3
10   2  sensor2   45.6
11   1  sensor1   12.3
12   2  sensor2   45.6
13   1  sensor1   12.3
14   2  sensor2   45.6
15   1  sensor1   12.3
16   2  sensor2   45.6
17   1  sensor1   12.3
18   2  sensor2   45.6
19   2  sensor2   45.6
20   1  sensor1   12.3
21   2  sensor2   45.6
22   2  sensor2   45.6
23   1  sensor1   12.3
24   1  sensor1   12.3
25   2  sensor2   45.6
26   1  sensor1   12.3
27   2  sensor2   45.6
28   1  sensor1   12.3
29   2  sensor2   45.6